<a href="https://colab.research.google.com/github/hihohub/umbrella-networks/blob/main/repository.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Umbrella networks
- copyright 2021 by James Smith
- contact - derekjsmit@gmail.com

Demonstration instructions
- How to load an online image repository into an umbrella network.
- You will load the cifar 10 dataset, which contains small 32x32 images.
- First, you will train a single neural net in keras. For simplicity, the model will train for only 10 epochs. The model gets a low accuracy.
- Then, you will compare that with a multi-model hierarchical umbrella network. Although it trains each model for only 10 epochs, it achieves a respectable accuracy on the same data.
- To start, run the first large cell containing the umbrella code, then scroll down 1333 lines to the experiment.
- Run each cell of the experiment, which is fairly quick.

Umbrella network instructions<br/>
- The umbrella code was designed for reading a local folder and automatically constructing the hierarchical network from the implicit hierarchy in the subfolders.
- If have images in folders, initialize with images_folder = path. Umbrella network constructs tree by itself. Refer to examples.
- However, it also has functions for reading the data from an online repository and building the tree node by node (since the data has no folder structure).
- If loading data from repository, initialize with images_folder = None, then use repo functions. Must first preprocess the data (normalize, reshape). Only process x data, not labels. Umbrella network constructs labels by itself. However, subdivide classes by label, then manually construct tree with repo functions. Finally, init_from_repo(). Refer to examples.
- In repo mode, umbrella network throws away labels and makes its own. For example, if divide MNIST into odd and even numbers, each subset will have labels 0-4. Hence, anything other than flat topology requires that the umbrella network form new labels.




%%%%%%%%%%%%%%%%%%<br/>
UMBRELLA NETWORK<br/>
%%%%%%%%%%%%%%%%%%<br/>

In [1]:
! wget https://raw.githubusercontent.com/hihohub/umbrella-networks/main/umbrella.py

--2022-04-15 04:36:07--  https://raw.githubusercontent.com/hihohub/umbrella-networks/main/umbrella.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48219 (47K) [text/plain]
Saving to: ‘umbrella.py’

umbrella.py         100%[===================>]  47.09K  --.-KB/s    in 0.001s  

2022-04-15 04:36:08 (83.6 MB/s) - ‘umbrella.py’ saved [48219/48219]



In [2]:
import umbrella
from umbrella import Umbrella_Network,Umbrella_Label,Umbrella_Node,Joint_Probability_Network

In [3]:
import os
import sys
import numpy as np
import math
import random
from PIL import Image, ImageOps, ImageChops
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, add, BatchNormalization, Dropout, AveragePooling2D, GlobalAveragePooling2D, SeparableConv2D
from tensorflow.keras.utils import to_categorical
from tensorflow.python.framework.ops import disable_eager_execution
from tensorflow.keras import backend as backend
from keras.callbacks import ReduceLROnPlateau
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import pickle
import copy
import time

### cifar10
- https://www.cs.toronto.edu/~kriz/cifar.html
1. airplane
2. automobile
3. bird
4. cat
5. deer
6. dog
7. frog
8. horse
9. ship
10. truck

In [4]:
import tensorflow
(xtrain,ytrain),(xval,yval) = tensorflow.keras.datasets.cifar10.load_data()
print(len(xtrain))
print(xtrain.shape)
print(len(ytrain))
print(ytrain.shape)
print(len(xval))
print(xval.shape)
print(len(yval))
print(yval.shape)
print(np.unique(yval))
xtrain = xtrain[0:10000,:]
print(xtrain.shape)
xtrain = xtrain / 255
print(xtrain[0])

170508288/170498071 [==============================] - 13s 0us/step
50000
(50000, 32, 32, 3)
50000
(50000, 1)
10000
(10000, 32, 32, 3)
10000
(10000, 1)
[0 1 2 3 4 5 6 7 8 9]
(10000, 32, 32, 3)
[[[0.23137255 0.24313725 0.24705882]
  [0.16862745 0.18039216 0.17647059]
  [0.19607843 0.18823529 0.16862745]
  ...
  [0.61960784 0.51764706 0.42352941]
  [0.59607843 0.49019608 0.4       ]
  [0.58039216 0.48627451 0.40392157]]

 [[0.0627451  0.07843137 0.07843137]
  [0.         0.         0.        ]
  [0.07058824 0.03137255 0.        ]
  ...
  [0.48235294 0.34509804 0.21568627]
  [0.46666667 0.3254902  0.19607843]
  [0.47843137 0.34117647 0.22352941]]

 [[0.09803922 0.09411765 0.08235294]
  [0.0627451  0.02745098 0.        ]
  [0.19215686 0.10588235 0.03137255]
  ...
  [0.4627451  0.32941176 0.19607843]
  [0.47058824 0.32941176 0.19607843]
  [0.42745098 0.28627451 0.16470588]]

 ...

 [[0.81568627 0.66666667 0.37647059]
  [0.78823529 0.6        0.13333333]
  [0.77647059 0.63137255 0.10196078]


In [5]:
# make an array for each class (discard y labels, umbrella network makes labels)
x0 = [xtrain[j] for j in range(0,len(xtrain)) if ytrain[j] == 0]
x1 = [xtrain[j] for j in range(0,len(xtrain)) if ytrain[j] == 1]
x2 = [xtrain[j] for j in range(0,len(xtrain)) if ytrain[j] == 2]
x3 = [xtrain[j] for j in range(0,len(xtrain)) if ytrain[j] == 3]
x4 = [xtrain[j] for j in range(0,len(xtrain)) if ytrain[j] == 4]
x5 = [xtrain[j] for j in range(0,len(xtrain)) if ytrain[j] == 5]
x6 = [xtrain[j] for j in range(0,len(xtrain)) if ytrain[j] == 6]
x7 = [xtrain[j] for j in range(0,len(xtrain)) if ytrain[j] == 7]
x8 = [xtrain[j] for j in range(0,len(xtrain)) if ytrain[j] == 8]
x9 = [xtrain[j] for j in range(0,len(xtrain)) if ytrain[j] == 9]
print(len(x0))
print(len(x1))
print(len(x2))
print(len(x3))
print(len(x4))
print(len(x5))
print(len(x6))
print(len(x7))
print(len(x8))
print(len(x9))

1005
974
1032
1016
999
937
1030
1001
1025
981


- #########################
- First test a single neural net without hierarchical umbrella
- To do so, just set "levels" = 1 in umbrella network
- Same as a regular single neural net with a softmax over 10 classes
- #########################

In [6]:
network_type = 'softmax'
images_folder = None # not saving any data for demonstration
serial_file = ""
models_folder = ""
target_image_size = 32 # cifar image size
channels = 3
levels = 1 # levels for network without hierarchical umbrella
valid_ratio = 0.2 # validation set
test_ratio = 0.0 # test set
initialize_now = False # delay initialization until construct tree node by node
CIFAR10 = Umbrella_Network(network_type,images_folder,serial_file,models_folder,target_image_size,channels,levels,valid_ratio,test_ratio,initialize_now)


warning - no source image folder specified
instead, you must construct tree manually with repository data
warning - could not find models folder


In [7]:
# construct tree node by node for cifar repository
root = CIFAR10.set_repo_root("root")
zeros = CIFAR10.add_repo_node(root,"zero")
ones = CIFAR10.add_repo_node(root,"one")
twos = CIFAR10.add_repo_node(root,"two")
threes = CIFAR10.add_repo_node(root,"three")
fours = CIFAR10.add_repo_node(root,"four")
fives = CIFAR10.add_repo_node(root,"fives")
sixes = CIFAR10.add_repo_node(root,"sixes")
sevens = CIFAR10.add_repo_node(root,"sevens")
eights = CIFAR10.add_repo_node(root,"eights")
nines = CIFAR10.add_repo_node(root,"nines")
CIFAR10.add_repo_nodes(zeros,x0,"zero")
CIFAR10.add_repo_nodes(ones,x1,"one")
CIFAR10.add_repo_nodes(twos,x2,"two")
CIFAR10.add_repo_nodes(threes,x3,"three")
CIFAR10.add_repo_nodes(fours,x4,"four")
CIFAR10.add_repo_nodes(fives,x5,"five")
CIFAR10.add_repo_nodes(sixes,x6,"six")
CIFAR10.add_repo_nodes(sevens,x7,"seven")
CIFAR10.add_repo_nodes(eights,x8,"eight")
CIFAR10.add_repo_nodes(nines,x9,"nine")

In [8]:
# initialize
CIFAR10.init_from_repo()
#CIFAR10.save_just_images() # for demonstration, not saving files

In [9]:
# function parameter for custom model rather than default model
def factory(node):
    node.model = Sequential()
    node.model.add(Conv2D(16,(3,3),activation='relu',input_shape=node.trainX[0].data.shape))
    node.model.add(Conv2D(32,(3,3),activation='relu'))
    node.model.add(Conv2D(64,(3,3),activation='relu'))
    node.model.add(Conv2D(128,(3,3),activation='relu'))
    node.model.add(Conv2D(256,(3,3),activation='relu'))
    node.model.add(Conv2D(512,(3,3),activation='relu'))
    node.model.add(MaxPool2D(pool_size=(2,2)))
    node.model.add(Flatten())
    node.model.add(Dense(32,activation='relu'))
    node.model.add(Dense(16,activation='relu'))
    node.model.add(Dense(len(node.children),activation='softmax'))
    node.model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
verbose = True
CIFAR10.make_models(factory,verbose)

making model for root
root
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 16)        448       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        4640      
                                                                 
 conv2d_2 (Conv2D)           (None, 26, 26, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 128)       73856     
                                                                 
 conv2d_4 (Conv2D)           (None, 22, 22, 256)       295168    
                                                                 
 conv2d_5 (Conv2D)           (None, 20, 20, 512)       1180160   
                                                                 
 max_pooling2d (MaxPooling2D 

In [10]:
# default epochs 5, to change set eps=x
CIFAR10.train_models(vrb=True,aug=True,eps=1)

training model for root
trainX 8001
trainY 8001
251/251 [==============================] - 17s 21ms/step - loss: 2.2827 - accuracy: 0.1199


In [11]:
CIFAR10.get_training_accuracy()
CIFAR10.predict_validation_set() # set verbose=True to view all the images (2000 of them)
CIFAR10.predict_validation_top5()

average training accuracy 0.119860
average training loss 2.282684
1999

average validation accuracy 0.138069
top 5 accuracy 0.59


- #############################################################
- Compare with multi-level umbrella
- Umbrella with two levels gets much higher scores
- #############################################################

In [12]:
network_type = 'softmax'
images_folder = None
serial_file = ""
models_folder = ""
target_image_size = 32
channels = 3
levels = 2 # multiple levels
valid_ratio = 0.2
test_ratio = 0.0
initialize_now = False
CIFAR10_UMBRELLA = Umbrella_Network(network_type,images_folder,serial_file,models_folder,target_image_size,channels,levels,valid_ratio,test_ratio,initialize_now)


warning - no source image folder specified
instead, you must construct tree manually with repository data
warning - could not find models folder


In [13]:
root = CIFAR10_UMBRELLA.set_repo_root("root")
animals = CIFAR10_UMBRELLA.add_repo_node(root,"animals")
objects = CIFAR10_UMBRELLA.add_repo_node(root,"objects")
airplanes = CIFAR10_UMBRELLA.add_repo_node(objects,"airplane")
autos = CIFAR10_UMBRELLA.add_repo_node(objects,"automobile")
birds = CIFAR10_UMBRELLA.add_repo_node(animals,"bird")
cats = CIFAR10_UMBRELLA.add_repo_node(animals,"cat")
deer = CIFAR10_UMBRELLA.add_repo_node(animals,"deer")
dogs = CIFAR10_UMBRELLA.add_repo_node(animals,"dog")
frogs = CIFAR10_UMBRELLA.add_repo_node(animals,"frog")
horses = CIFAR10_UMBRELLA.add_repo_node(animals,"horse")
ships = CIFAR10_UMBRELLA.add_repo_node(objects,"ship")
trucks = CIFAR10_UMBRELLA.add_repo_node(objects,"truck")
CIFAR10_UMBRELLA.add_repo_nodes(airplanes,x0,"airplane")
CIFAR10_UMBRELLA.add_repo_nodes(autos,x1,"auto")
CIFAR10_UMBRELLA.add_repo_nodes(birds,x2,"bird")
CIFAR10_UMBRELLA.add_repo_nodes(cats,x3,"cat")
CIFAR10_UMBRELLA.add_repo_nodes(deer,x4,"deer")
CIFAR10_UMBRELLA.add_repo_nodes(dogs,x5,"dog")
CIFAR10_UMBRELLA.add_repo_nodes(frogs,x6,"frog")
CIFAR10_UMBRELLA.add_repo_nodes(horses,x7,"horse")
CIFAR10_UMBRELLA.add_repo_nodes(ships,x8,"ship")
CIFAR10_UMBRELLA.add_repo_nodes(trucks,x9,"truck")

In [14]:
CIFAR10_UMBRELLA.init_from_repo()

In [15]:
def factory(node):
    node.model = Sequential()
    node.model.add(Conv2D(16,(3,3),activation='relu',input_shape=node.trainX[0].data.shape))
    #node.model.add(MaxPool2D(pool_size=(2,2)))
    node.model.add(Conv2D(32,(3,3),activation='relu'))
    #node.model.add(MaxPool2D(pool_size=(2,2)))
    node.model.add(Conv2D(64,(3,3),activation='relu'))
    node.model.add(MaxPool2D(pool_size=(2,2)))
    node.model.add(Conv2D(128,(3,3),activation='relu'))
    node.model.add(MaxPool2D(pool_size=(2,2)))
    node.model.add(Flatten())
    node.model.add(Dense(32,activation='relu'))
    node.model.add(Dense(len(node.children),activation='softmax'))
    node.model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
verbose = True
CIFAR10_UMBRELLA.make_models(factory,verbose)

making model for root
root
Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 30, 30, 16)        448       
                                                                 
 conv2d_7 (Conv2D)           (None, 28, 28, 32)        4640      
                                                                 
 conv2d_8 (Conv2D)           (None, 26, 26, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 11, 11, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 5, 5, 128)        0         
 2D)                      

In [16]:
CIFAR10_UMBRELLA.train_models(vrb=True,aug=True,eps=1)

training model for root
trainX 8001
trainY 8001
251/251 [==============================] - 2s 5ms/step - loss: 0.6730 - accuracy: 0.6012
training model for animals
trainX 4813
trainY 4813
151/151 [==============================] - 1s 6ms/step - loss: 1.7375 - accuracy: 0.2367
training model for objects
trainX 3188
trainY 3188
100/100 [==============================] - 1s 7ms/step - loss: 1.3175 - accuracy: 0.3479


In [17]:
CIFAR10_UMBRELLA.get_training_accuracy()
CIFAR10_UMBRELLA.predict_validation_set()
CIFAR10_UMBRELLA.predict_validation_top5()

average training accuracy 0.395231
average training loss 1.242663
1999

average validation accuracy 0.225613
top 5 accuracy 0.64
